# Easy-Trustflow快速体验

## 部署一个在线TEE服务
一个在线TEE服务需要对外提供远程证明服务，并基于远程证明与它的客户端建立安全的通信连接。我们可以集成Ra Proxy和Trustflow Envoy这两个sidecar来快速实现远程证明服务和加密通信功能，无需侵入应用本体。

一个典型的交互流程如下:

![online_app](../images/online_app.jpg)

整个CVM POD我们视为一个完整的TEE APP。

1. 用户向TEE APP发起远程认证请求，请求中携带一个随机的挑战值nonce。请求入口为Trustflow Envoy。

2. Trustflow Envoy将请求路由给Ra Proxy。

3. Ra Proxy生成远程认证报告，报告中携带nonce和APP公钥的哈希值。

4. Trustflow Envoy转发Ra Proxy的response。

5. 用户访问可信根机构对该远程认证报告进行验签，校验TEE APP的度量值、APP公钥、nonce值等是否符合预期，校验通过后用户就认为这个TEE APP是可信的，并且获取到了可信的APP公钥。

6. 用户使用数据密钥加密用户数据，用APP公钥加密数据密钥。

7. 用户发送加密请求，请求入口为Trustflow Envoy。

8. Trustflow Envoy识别到这是一个数字信封加密请求，使用APP私钥解密。

9. Trustflow Envoy将解密后的请求转发给APP Container。

10. APP Container处理业务逻辑，返回结果。

11. Trustflow Envoy使用数据密钥加密返回数据。

12. Trustflow Envoy将加密后的数据返回给用户。

13. 用户使用数据密钥解密返回数据。


这里我们提供一个简单示例作为参考。应用本体是一个简单的http服务。部署者为[快速上手](../quick_start/step0.ipynb)中的carol。

### 准备工作
请参考[快速上手](../quick_start/step0.ipynb)中步骤准备机构公钥。

进入到[Trustflow代码仓库](https://github.com/asterinas/trustflow)的examples/online-sample目录。

创建shared/deploy目录，carol将自己的公私钥作为APP公私钥放置在shared/deploy目录下。例如examples/online-sample/shared/deploy/carol.crt和examples/online-sample/shared/deploy/carol.key。

创建shared/log目录，sidecar的日志将会输出到该目录下。

### 使用Docker Compose部署服务
进入examples/online-sample目录，根据您所在的TEE平台选择对应的配置文件。

在真实TEE环境中，请先参照[快速上手步骤四](../quick_start/step4.ipynb)中的方式检查CVM环境并安装对应的驱动。

```bash
# sim模式
docker compose -f docker-compose-sim.yml up
```
```bash
# tdx模式
docker compose -f docker-compose-tdx.yml up
```
```bash
# csv模式
docker compose -f docker-compose-csv.yml up
```

### 访问TEE服务
进入examples/online-sample/client目录。

1. 安装依赖
```bash
pip install -r requirements.txt
```

2. 使用jwe客户端加密访问TEE服务
```bash
python3 jwe_client.py
```
您将得到类似以下的输出内容：
```text
Response: {'protected': 'eyJhbGciOiJSU0EtT0FFUC0yNTYiLCJlbmMiOiJBMTI4R0NNIn0', 'encrypted_key': '', 'iv': 'C5Fu4JOLZYFTr8NG', 'ciphertext': 'fQrNaUf_D0pyI06vXWyOQJpyMqCj7CwxqqM', 'tag': 'AS7wxWp7sItHeVr26eLshQ', 'aad': ''}
Response Raw: I got your message: hello
```
这说明内容为"hello"的加密请求已经被Trustflow Envoy解密，并转发给APP Container处理。APP Container返回了一条内容为"I got your message: hello"的数据并由Trustflow Envoy加密返回给客户端。客户端解密得到了Response Raw。

## 部署一个离线TEE任务
一个离线TEE任务与[快速上手步骤四](../quick_start/step4.ipynb)中功能类似。我们可以通过集成Data Capsule Proxy来实现数据密钥的获取、数据的加解密、计算结果数据密钥的上传注册等功能。在APP本体中无需关心TEE本身的细节。

一个典型的交互流程如下，APP Container借助Data Capsule Proxy获取计算数据并进行计算：

![offline_app](../images/offline_app.jpg)

1. APP Container向Data Capsule Proxy请求计算数据。请求中需要指定加密数据的路径（可以是本地路径或者OSS路径）、数据密钥在Capsule Manager中的相关信息以及解密后数据放置的路径。具体接口请见[Data Capsule Proxy接口](https://github.com/secretflow/secure-data-capsule-apis/blob/main/secretflowapis/v2/sdc/data_capsule_proxy/data_capsule_proxy.proto)。

2. Data Capsule Proxy会生成远程认证报告，补充完整请求内容，然后向Capsule Manager请求数据密钥。

3. Capsule Manager校验远程认证报告。

4. Capsule Manager根据授权策略进行访问鉴权。

5. 鉴权通过后Capsule Manager使用APP公钥加密数据密钥。

6. Capsule Manager将加密的数据密钥返回给Data Capsule Proxy。

7. Data Capsule Proxy使用APP私钥解密数据密钥。

8. Data Capsule Proxy用数据密钥解密计算数据。其中加密计算数据的路径在第1步中由APP Container指定，可以是本地路径或者OSS路）。

9. Data Capsule Proxy将解密后的计算数据保存至指定路径。该路径也在第1步中由APP Container指定，是一个本地加密磁盘中的路径。

10. APP Container从本地加密磁盘中读取计算数据，进行计算。

在计算完成之后，APP Container还可以借助Data Capsule Proxy将计算结果和新的数据密钥上传至Capsule Manager。

![offline_app_result](../images/offline_app_result.jpg)

1. APP Container将结果数据写入加密磁盘。

2. APP Container向Data Capsule Proxy请求加密上传结果数据。请求中需要指定结果数据在加密磁盘中的路径、数据密钥、结果数据的数据血缘关系以及加密后上传的路径（可以是本地路径或者OSS路径）。

3. Data Capsule Proxy使用APP Container指定的数据密钥加密结果数据，如果没有指定数据密钥则会随机生成一个新的数据密钥。加密完成后将加密结果数据上传至指定位置。

4. Data Capsule Proxy使用Capeule Manager的公钥加密数据密钥。

5. Data Capsule Proxy生成远程认证报告，补充完整请求内容，然后向Capsule Manager请求注册结果数据密钥。

6. Capsule Manager校验远程认证报告。

7. Capsule Manager使用自己的私钥解密数据密钥。

8. Capsule Manager根据请求中的数据血缘关系生成该数据密钥的授权策略并存储。


这里我们以一个psi的简单示例作为参考。部署者为[快速上手](../quick_start/step0.ipynb)中的carol。alice和bob将自己的数据授权给carol使用。

### 准备工作
请参考[快速上手](../quick_start/step0.ipynb)中步骤准备机构公钥。

参考[快速上手步骤一](../quick_start/step1.ipynb)中步骤部署Capsule Manager。

参考[快速上手步骤二](../quick_start/step2.ipynb)中步骤让alice和bob分别进行数据加密、数据密钥上传。

参考[快速上手步骤三](../quick_start/step3.ipynb)中步骤让alice和bob分别对carol进行数据密钥授权。

进入到[Trustflow代码仓库](https://github.com/asterinas/trustflow)的examples/offline-sample目录。

创建shared/deploy目录，carol将自己的公私钥作为APP公私钥放置在shared/deploy目录下。例如examples/offline-sample/shared/deploy/carol.crt和examples/offline-sample/shared/deploy/carol.key。

创建shared/log目录，sidecar的日志将会输出到该目录下。

### 准备加密数据集
进入到[Trustflow代码仓库](https://github.com/asterinas/trustflow)的examples/docker-compose/offline-sample目录，创建shared/data目录，carol将[快速上手步骤二](../quick_start/step2.ipynb)中得到的alice和bob的加密数据集alice.csv.enc和bob.csv.enc放到该目录下。

### 修改应用配置
将teeapp/psi.py中的_CAPSULE_MANAGER_ENDPOINT替换为实际部署的capsule manager的地址。示例中的capsule manager部署在本机的8888端口，因此访问了docker网关的地址。

### 使用Docker Compose部署服务
进入examples/online-sample目录，根据您所在的TEE平台选择对应的配置文件。

在真实TEE环境中，请先参照[快速上手步骤四](../quick_start/step4.ipynb)中的方式检查CVM环境并安装对应的驱动。

```bash
# sim模式
docker compose -f docker-compose-sim.yml up
```
```bash
# tdx模式
docker compose -f docker-compose-tdx.yml up
```
```bash
# csv模式
docker compose -f docker-compose-csv.yml up
```

您将在shared/data目录看到求交的结果。

## 部署两个安全互通的TEE应用
如果一项任务需要两个或以上不同的应用互相通信。我们可以借助Data Capsule Proxy获取通信组密钥，然后借助Trustflow Envoy建立双向的TLS连接。

我们认为通信数据可以视作输入数据的衍生，所以通信组密钥是来自数据密钥派生的。

两个TEE应用建立双向TLS连接的流程如下：

![interact_app](../images/interact_app.jpg)

1. TEE APP A和TEE APP B的Data Capsule Proxy在启动时向Capsule Manager请求通信组密钥。请求中指定数据密钥的相关信息，并携带远程认证报告和APP公钥。

2. Capsule Manager校验远程认证报告。

3. Capsule Manager根据授权策略进行访问鉴权。

4. 鉴权通过后，Capsule Manager使用数据密钥派生通信组密钥。

5. Capsule Manager使用TEE APP的公钥加密通信组密钥。

6. Capsule Manager返回加密的通信组密钥。

7. Data Capsule Proxy使用APP私钥解密得到通信组密钥。

8. Data Capsule Proxy将通信组密钥写入加密磁盘。

9. Trustflow Envoy读取加密磁盘中的通信组密钥并与其他Trustflow Envoy建立mTLS连接。后续APP Container之间的交互通过Trustflow Envoy代理完成。

我们给出了一个简单的示例，[快速上手](../quick_start/step0.ipynb)中的bob和carol分别部署一个TEE应用，为一个简单的http服务。alice将数据密钥授权给bob和carol，然后bob和carol通过Data Capsule Proxy获取通信组密钥，并使用Trustflow Envoy建立双向TLS连接。

### 准备工作
请参考[快速上手](../quick_start/step0.ipynb)中步骤准备机构公钥。

参考[快速上手步骤一](../quick_start/step1.ipynb)中步骤部署Capsule Manager。

参考[快速上手步骤二](../quick_start/step2.ipynb)中步骤让alice进行数据密钥上传，只需alice上传。

bob和carol分别进入到[Trustflow代码仓库](https://github.com/asterinas/trustflow)的examples/interact-sample/bob和examples/interact-sample/carol目录。

创建shared/deploy目录，bob和carol分别将自己的公私钥作为APP公私钥放置在shared/deploy目录下。

创建shared/log目录，sidecar的日志将会输出到该目录下。

### alice授权给bob和carol
在capsule managaer sdk的配置文件alice.yaml中配置add_data_rule字段。请将grantee_party_ids字段替换成bob和carol的party_id。
```yaml
add_data_rule:
  # (required) str
  scope: default
  # (required) str
  data_uuid: breast_cancer_alice
  # (required) dict
  rule:
    # (required) str
    rule_id: alice_rule_id2
    # (required) List[str]
    grantee_party_ids: 
       # Replace with real party_id
       - xxxx
       - xxxx
    # (required) List[str]
    columns: 
      - "*"
    # (optional) List[str]
    global_constraints:
    # (required) List[dict]
    op_constraints:
      - 
        # (required) str
        op_name: interact
        # (optional) List[str]
        constraints:
```

alice执行
```bash
cms --config-file alice.yaml add-data-rule
```
成功后，alice就授权给bob和carol进行interact操作了。

### 修改应用配置
修改docker-compose-xxx.yml文件中data-capsule-proxy的command参数，将--cm_endpoint替换成真实的Capsule Manager地址。

示例中的capsule manager部署在本机的8888端口，因此访问了docker网关的地址。

### 使用Docker Compose部署服务
bob和carol分别进入examples/interact-sample/bob和examples/interact-sample/carol目录，根据您所在的TEE平台选择对应的配置文件。

在真实TEE环境中，请先参照[快速上手步骤四](../quick_start/step4.ipynb)中的方式检查CVM环境并安装对应的驱动。

```bash
# sim模式
docker compose -f docker-compose-sim.yml up
```
```bash
# tdx模式
docker compose -f docker-compose-tdx.yml up
```
```bash
# csv模式
docker compose -f docker-compose-csv.yml up
```

在bob端可以看到以下输出：
```text
response:  Hi, I'm Carol. I got your message: {"message": "Hello from Bob!"}
```

在carol端可以看到以下输出：
```text
response:  Hi, I'm Bob. I got your message: {"message": "Hello from Carol!"}
``
说明两个APP已经能够通过tls安全通信了。